In [1]:
# Calculate fission Spectra weighting 
import csv    # Library to read & process csv-formatted files
import sys    # Library to parse command-line arguments
import os
from math import sqrt, sinh, pi, exp
import numpy as np

rootdir=os.path.abspath(os.getcwd())

# Read in the spectra from file
fname = rootdir+"/E_fiss.csv"   # Fission spectra input file
e_bins=[]
fiss=[]
err=[]
try: 
    ifile = open(fname, 'r') 

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        e_bins.append(float(split_list[0].strip()))
        fiss.append(float(split_list[1].strip()))
        err.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)
    
np.asarray(fiss)
fiss=fiss/np.sum(fiss)

In [2]:
# Read in the bin structure from file
rootdir_235=os.path.abspath(os.path.join(rootdir, os.pardir))+'/Validation/235Watt'
rootdir_238=os.path.abspath(os.path.join(rootdir, os.pardir))+'/Validation/238Watt'
fname = rootdir_235+"/Bins.csv"   # Bin structure input file
bins=[]

try: 
    ifile = open(fname, 'r') 
    
    # Skip header line
    line=ifile.next()

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        bins.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)

#print len(bins)
#print bins

In [7]:
def Build_Weighted_FPs(bins,fw,datadir):
    one=0
    two=0
    y={}
    err={}
    w=[]
    fy=[]

    # Use weighted fission spectra to generate mass yield curves (sum yields?)
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            if file=="GEF.dat":
                #print subdir, file
                weight = 0
                one+=1
                try: 
                    ifile = open(subdir+'/'+file, 'r') 
                    for line in ifile:
                        if line[:7] == ' formed':
                            split_list=line.split(' ')
                            for i in range(0,len(bins)):
                                if bins[i] == float(split_list[8].strip()):
                                    two+=1
                                    weight=fw[i]
                                    #print "Energy =", bins[i]
                                    #print "Fractional weight=",weight
                        if line[:27] =='--- Mass-yield distribution':
                            for i in range(0,6):
                                line=ifile.next()
                            while line[:22] != '        </Mass_yields>':
                                split_list=line.split()
                                if not split_list[0] in y.keys():  # this is the first time we've encountered this mass A
                                # let's initialize our dictionaries
                                    y[split_list[0]] = float(split_list[2])*weight
                                    err[split_list[0]] = (float(split_list[3])*weight)**2
                                else:  # we've encoutered this mass before, no need to initialize, just add to it
                                    y[split_list[0]] += float(split_list[2])*weight
                                    err[split_list[0]] += (float(split_list[3])*weight)**2

                                if int(split_list[0])==160:
                                    w.append(weight)
                                    fy.append(float(split_list[2]))
                                    print float(split_list[3])
                                line=ifile.next()
                    # Close the file
                    ifile.close()
                except IOError as e:
                    print "I/O error({0}): {1}".format(e.errno, e.strerror)
                    print "File not found was: {0}".format(fname)
    #print y
    #print y.keys()

    if one!=two or one!=len(bins):
        print "/n/nError: Not all of the files from {} were imported correctly./n/n".format(datadir)
        print one, two, len(bins)
        
    return y, err

In [13]:
#y={}
#err={}
#(y,err)=Build_Weighted_FPs(bins,fiss,rootdir_235)

y_235={}
err_235={}
f_235=0.93155
(y_235,err_235)=Build_Weighted_FPs(bins,fiss*f_235,rootdir_235)

print "\n\n"
for A in y_235.keys():
    if A=='160':
        print err[A]
    err[A]=sqrt(err_235[A])

#y_238={}
#err_238={}
#f_238=1-f_235
#(y_238,err_238)=Build_Weighted_FPs(bins,fiss*f_238,rootdir_238)

#for A in y_238.keys():
#    err[A]=sqrt(err_238[A])

#for A in y.keys():
#    try:
#        y[A]=y_235[A]+y_238[A]
#        err[A]=sqrt(err_235[A]**2+err_238[A]**2)
#    except KeyError as e:
#        print "Key error: {0} not found for both isotopes.".format(A)

0.0002
0.00326
0.0003
0.0005
0.0005
0.0006
0.0026
0.0007
0.00319
0.00512
0.0008
0.00531
0.00512
0.0004
0.0004
0.0003
0.0042
0.00305
0.0004
0.0028
0.0004
0.0003
0.00441
0.00291
0.00349
0.0019
0.0004
0.00465
0.00547
0.0002
0.00616
0.0032
0.0003
0.00357
0.0008
0.0003
0.0012
0.0065
0.0008
0.00422
0.0009
0.0002
0.0002
0.0008
0.0009
0.0004



0.000696144118504


In [5]:
# Output the result
outfn="ETA_sum_fy.csv"

def writeout(outfn,outl):  # function to write our output in csv format
    ofile = open(outfn,"w")

    for out in outl:
        ofile.write(str(out[0])
                    + "," + str(out[1])
                    + "," + str(out[2])
                    + "," + str(out[3])
                    + "," + str(out[4])
                    + "\n")

    ofile.close()

outl=[]
for A in y.keys():
    outl.append((A,0,y[A],err[A]," "))
writeout(outfn,outl)